In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [0]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  regex = re.compile('[^A-Za-zА-Яа-я]')
  cleantext = regex.sub(' ', cleantext)
  return cleantext

### Text

In [30]:
##вроде работает для этого класса, но делает лист с листами
def text(url):
    page = requests.get('https://vc.ru/' + str(url))
    soup = BeautifulSoup(page.text, 'html5lib')
    text = ''
    for item in soup.find_all('div',  class_="layout--с block-html"):
        text += item.text
    return text

In [115]:
def document(id):
    page = requests.get('https://vc.ru/' + str(id) )
    soup = BeautifulSoup(page.text, 'html5lib')
    info = []
    text =[]
    lol = []
    if soup.find("h1", class_="content-header__title") and 'Вакансия' not in soup.find('title').text:
        #titledone
        info.append(soup.find("h1", class_="content-header__title").text.strip())
        #text
        text = ' '
        for i in soup.find_all('div',  class_="layout--с block-html"):
            text += i.text.strip()
        text += soup.find_all('div', class_="content content--full")[0].find('div', class_="layout--a").text.strip()
        for i in soup.find_all("li"):
            text += i.text
        info.append(text)
        #yeardone
        info.append(soup.find("time", class_= "time").text.split()[2])
        #sectiondone
        info.append(soup.find("div", class_=
                              "content-header-author__name").text.strip())
        #votesdone
        a = soup.find("span", class_="vote__value__v vote__value__v--real").text
        if a[0] == '–':
            b = int(a[1:])
            info.append(-b)
        else:
            info.append(int(a))
        #viewsdone
        info.append(soup.find("span", class_="views__value").text.replace(u'\xa0', u''))
        #bookmarksdone
        if soup.find('div',
                        class_="favorite_marker favorite_marker--type-content favorite_marker--non_zero"):
            info.append(int(soup.find('div',
                        class_="favorite_marker favorite_marker--type-content favorite_marker--non_zero").text.split()[0]))
        else:
            info.append(0)
        #commentsdone
        comment1 = soup.find_all("div", class_="comments__item__text")
        comments = ' '
        for i in range(len(comment1)):
            if comment1[i].text != 'Комментарий удален':
                comments += '~#' + comment1[i].text
        info.append(comments)
    else:
        info.append('NaN')
    print(id)
    return info

In [111]:
a = map(document, [110342, 110343])

In [112]:
len(a)

TypeError: object of type 'map' has no len()

In [113]:
df = pd.DataFrame(a, columns=['title', 'text', 'date', 'section', 'votes', 'views', 'bookmarks', 'comments'])

110342
110343


In [114]:
df

,title,text,date,section,votes,views,bookmarks,comments
0,Как сервис по подбору психологов Alter из агре...,Раньше Alter работал в формате агрегатора: кл...,16:51,\n Сервисы,33.0,3573,126.0,~#Скайенг с психологами по скайпу! Срочно под...
1,NaN,None,None,None,NaN,None,NaN,None


In [0]:
i = 239400
for j in range(239500, 351001, 100):
    from multiprocessing import Pool
    with Pool(100) as p:
        b = p.map(document, np.arange(i, j))
    try:
        df = pd.DataFrame(b, columns=['title', 'text', 'date', 'hubs', 'tags', 'votes', 'views', 'bookmarks', 'comments'])
        df.to_feather(str(i) + "-" + str(j) + '.feather')
        from google.colab import files
        files.download(str(i) + "-" + str(j) + '.feather')
        print('done', i, j)
    except:
        print('ooops')
    i += 100

In [0]:
df.to_csv('vcru.csv')
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
files.download('vcru.csv')